In [1]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')

## Simplex Method

In [2]:
#Currently the code is for less than equal to type constraints problem

objective="max"
m=2#no of equations
n=2#no of variables
a=[[10.0,20.0,'<='],[8.0,8.0,"<="]]
b=[120.0,80.0]
Coeff=[12.0, 16.0]

In [3]:
def convert_to_maximization_type(a,b,Coeff,m,n,objective):
    if objective != 'max':
        Coeff=[-Coeff[i] for i in range(n)]
    for i in range(m):
        if a[i][-1] != '<=':
            for j in range(n):
                a[i][j]=-a[i][j]
            b[i]=-b[i]
    return a,b,Coeff,m,n
                
                
    

In [4]:


#1
def initialize_simplex_table(a, b, m, n, Coeff,objective):
    convert_dict={'x'+str(i+1):float for i in range(n)}
    jconvert_dict={'x'+str(i+1):float for i in range(n)}
    a,b,Coeff,m,n=convert_to_maximization_type(a,b,Coeff,m,n,objective)
    Cj_dict={'x'+str(i+1):[Coeff[i]] for i in range(n)}
    Zj_dict={'x'+str(i+1):[0] for i in range(n)}
    for i in range(m):
        Cj_dict['s'+str(i+1)]=[0]
        Zj_dict['s'+str(i+1)]=[0]
        convert_dict['s'+str(i+1)]=float
        jconvert_dict['s'+str(i+1)]=float
    Cj_dict['sol']=0
    Zj_dict['sol']=0   
    convert_dict['sol']=float
    jconvert_dict['sol']=float
    #Cj_dict.extend({0 for i in range(m)])
    #Zj=[0 for i in range(m+n)]
    data = {'CBi':[0 for i in range(m)]} 
    convert_dict['CBi']=float
    data['basic_var']=['s'+str(i+1) for i in range(m)]
    convert_dict['basic_var']=str
    for i in range(n):
        data['x'+str(i+1)]=[a[j][i] for j in range(m)]
    #augmenting data with slack variables
    for i in range(m):
        data['s'+str(i+1)]=[float(i==j) for j in range(m)]
    data['sol']=[b[j] for j in range(m)]
    data['ratio']=[0 for j in range(m)]
    convert_dict['ratio']=float
    df = pd.DataFrame(data, index=[str(i+1) for i in range(m)]) 
    Zj = pd.DataFrame(Zj_dict, index=['Zj']) 
    Cj = pd.DataFrame(Cj_dict, index=['Cj'])
    
    df = df.astype(convert_dict) 
    
    Zj = Zj.astype(jconvert_dict)
    Cj = Cj.astype(jconvert_dict)
    return df,Cj,Zj
  

In [5]:
#2 
def print_table(df, Cj, Zj):
    print("simplex table:")
    frames = [df, Cj, Zj]
    result = pd.concat(frames)
    
    print(result)
#     print("Cj's: ",Cj)
    
#     print("Zj's: ",Zj)
    
    
  
 
df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_table(df,Cj,Zj)

simplex table:
    CBi basic_var    x1    x2   s1   s2    sol  ratio
1   0.0        s1  10.0  20.0  1.0  0.0  120.0    0.0
2   0.0        s2   8.0   8.0  0.0  1.0   80.0    0.0
Cj  NaN       NaN  12.0  16.0  0.0  0.0    0.0    NaN
Zj  NaN       NaN   0.0   0.0  0.0  0.0    0.0    NaN


In [6]:
#3
def Zj_Cj(Cj, Zj):
    l=['x'+str(i+1) for i in range(n)]
    l.extend(['s'+str(i+1) for i in range(m)])
    return  [Zj[i]['Zj']-Cj[i]['Cj'] for i in l]


In [7]:
#4
def col_name(i,n):
    if (i+1)<=n:
        return 'x'+str(i+1)
    else:
        return 's'+str(i+1-n)
    
def get_key_column(zj_cj, m, n):
    val_min=zj_cj[0]
    idx_min='x1'  
    for i in range(m+n):
        if val_min>zj_cj[i]:
            val_min=zj_cj[i]
            idx_min=col_name(i,n)
    
    return idx_min
    


def get_key_row(df, key_col, m):   
    df['ratio']=df['sol']/df[key_col]
    val_min=999999999
    idx_min=0
    for i in range(m):
        if val_min>df['ratio'][i]>=0:
            val_min=df['ratio'][i]
            idx_min=i
    return df, str(idx_min+1), val_min




In [8]:
print("_____output_________")
zj_cj=Zj_Cj(Cj, Zj)
key_col=get_key_column(zj_cj, m, n)
df, key_row, min_ratio = get_key_row(df, key_col, m)
print(df)
print()
print("Key column is " )
print(df[key_col])
print()
print("Minimum ratio is ", min_ratio)
print()
print("Key row is ")
print(df.loc[key_row])
print()
print("Key element is ", df[key_col][key_row])


_____output_________
   CBi basic_var    x1    x2   s1   s2    sol  ratio
1  0.0        s1  10.0  20.0  1.0  0.0  120.0    6.0
2  0.0        s2   8.0   8.0  0.0  1.0   80.0   10.0

Key column is 
1    20.0
2     8.0
Name: x2, dtype: float64

Minimum ratio is  6.0

Key row is 
CBi            0
basic_var     s1
x1            10
x2            20
s1             1
s2             0
sol          120
ratio          6
Name: 1, dtype: object

Key element is  20.0


In [9]:
def modify_table(key_row, key_col, Cj, Zj,df):
    df_new = df.copy(deep=True)
    Zj_new = Zj.copy(deep=True)
    col_len=len(df[key_col])
    row_len=len(df.loc[key_row])
    p=key_elem=df[key_col][key_row]
    entering_var=key_col
    leaving_var=df['basic_var'][key_row]
    print("Entering variable is ", entering_var)
    print("Leaving variable is ", leaving_var)
    df_new['basic_var'][key_row]=entering_var
    
    
    col=['x'+str(i+1) for i in range(n)]
    col.extend(['s'+str(i+1) for i in range(m)])
    col.append('sol')
    row=list(df.index)
    
    for j in col:
        r=df[j][key_row]              
        for i in row:    
            if j==key_col:
                df_new[j][i]=(-df[j][i])/p                
            elif i==key_row:
                df_new[j][i]=df[j][i]/p               
            else:
                s=df[j][i]
                q=df[key_col][i] 
                df_new[j][i]=(p*s-q*r)/p
    #print(1/p)
    df_new[key_col][key_row]=1/p
    #df_new[key_col]=df[]
    df_new[leaving_var]  = df_new[entering_var]    
    df_new[entering_var] = 0
    df_new[entering_var][key_row]=1
    
    #swap the coefficients value
    df_new['CBi'][key_row]=Cj[key_col]['Cj']
    
    #modify zj
    for j in col:
        Zj_new[j]['Zj']=sum([df_new[j][i]*df_new['CBi'][i] for i in row]) 
        #Zj_new[j]['Zj']=16.0*0.05
    return df_new,Cj,Zj_new
    
    
    
    
    
    
    
    
    

In [10]:
df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df)
print_table(df_new,Cj,Zj_new)


Entering variable is  x2
Leaving variable is  s1
simplex table:
     CBi basic_var    x1    x2    s1   s2   sol  ratio
1   16.0        x2   0.5   1.0  0.05  0.0   6.0    6.0
2    0.0        s2   4.0   0.0 -0.40  1.0  32.0   10.0
Cj   NaN       NaN  12.0  16.0  0.00  0.0   0.0    NaN
Zj   NaN       NaN   8.0  16.0  0.80  0.0  96.0    NaN


In [11]:
#Check is optimal solution is reached or not
def is_optimal(zj_cj):
    val=min(zj_cj)
    if val>=0:
        print("Optimality is reached")
        return True
    else:
        print("Lets proceed to next iteration")
        return False

In [12]:
def simplex_method(a, b, m, n, Coeff,objective):    
    df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
    zj_cj=Zj_Cj(Cj, Zj)
    key_col=get_key_column(zj_cj, m, n)
    df, key_row, min_ratio = get_key_row(df, key_col, m)
    print_table(df,Cj,Zj)
    i=0
    while not is_optimal(zj_cj) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return
        df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df)
        
        df=df_new
        Zj=Zj_new
        
        if objective=="max":
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            print("value of objective function is ",-Zj['sol']['Zj'])
        
        zj_cj=Zj_Cj(Cj, Zj)
        key_col=get_key_column(zj_cj, m, n)
        df, key_row, min_ratio = get_key_row(df, key_col, m)  
        print_table(df,Cj,Zj)
        i+=1
        
    
    

In [13]:
simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var    x1    x2   s1   s2    sol  ratio
1   0.0        s1  10.0  20.0  1.0  0.0  120.0    6.0
2   0.0        s2   8.0   8.0  0.0  1.0   80.0   10.0
Cj  NaN       NaN  12.0  16.0  0.0  0.0    0.0    NaN
Zj  NaN       NaN   0.0   0.0  0.0  0.0    0.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s1
value of objective function is  96.0
simplex table:
     CBi basic_var    x1    x2    s1   s2   sol  ratio
1   16.0        x2   0.5   1.0  0.05  0.0   6.0   12.0
2    0.0        s2   4.0   0.0 -0.40  1.0  32.0    8.0
Cj   NaN       NaN  12.0  16.0  0.00  0.0   0.0    NaN
Zj   NaN       NaN   8.0  16.0  0.80  0.0  96.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s2
value of objective function is  128.0
simplex table:
     CBi basic_var    x1    x2   s1     s2    sol  ratio
1   16

In [14]:
#a
objective="max"
m=3#no of equations
n=3#no of variables
a=[[3,5,2,'<='],[4,4,4,"<="],[2,4,5,"<="]]
b=[60,72,100]
Coeff=[5,10,8]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1    x2   x3   s1   s2   s3    sol  ratio
1   0.0        s1  3.0   5.0  2.0  1.0  0.0  0.0   60.0   12.0
2   0.0        s2  4.0   4.0  4.0  0.0  1.0  0.0   72.0   18.0
3   0.0        s3  2.0   4.0  5.0  0.0  0.0  1.0  100.0   25.0
Cj  NaN       NaN  5.0  10.0  8.0  0.0  0.0  0.0    0.0    NaN
Zj  NaN       NaN  0.0   0.0  0.0  0.0  0.0  0.0    0.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s1
value of objective function is  120.0
simplex table:
     CBi basic_var   x1    x2   x3   s1   s2   s3    sol      ratio
1   10.0        x2  0.6   1.0  0.4  0.2  0.0  0.0   12.0  30.000000
2    0.0        s2  1.6   0.0  2.4 -0.8  1.0  0.0   24.0  10.000000
3    0.0        s3 -0.4   0.0  3.4 -0.8  0.0  1.0   52.0  15.294118
Cj   NaN       NaN  5.0  10.0  8.0  0.0  0.0  0.0    0.0        NaN
Zj   NaN       NaN  6.0  10.0  4.0  2.0  0.0  0.0  120.0        NaN
Lets proce

In [15]:
#b
objective="max"
m=4#no of equations
n=2#no of variables
a=[[2,1,'<='],[1,1,"<="],[1,0,"<="],[0,1,"<="]]
b=[1000,800,400,700]
Coeff=[4,3]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   s1   s2   s3   s4     sol  ratio
1   0.0        s1  2.0  1.0  1.0  0.0  0.0  0.0  1000.0  500.0
2   0.0        s2  1.0  1.0  0.0  1.0  0.0  0.0   800.0  800.0
3   0.0        s3  1.0  0.0  0.0  0.0  1.0  0.0   400.0  400.0
4   0.0        s4  0.0  1.0  0.0  0.0  0.0  1.0   700.0    inf
Cj  NaN       NaN  4.0  3.0  0.0  0.0  0.0  0.0     0.0    NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0     0.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s3
value of objective function is  1600.0
simplex table:
    CBi basic_var   x1   x2   s1   s2   s3   s4     sol  ratio
1   0.0        s1  0.0  1.0  1.0  0.0 -2.0  0.0   200.0  200.0
2   0.0        s2  0.0  1.0  0.0  1.0 -1.0  0.0   400.0  400.0
3   4.0        x1  1.0  0.0  0.0  0.0  1.0  0.0   400.0    inf
4   0.0        s4  0.0  1.0  0.0  0.0 -0.0  1.0   700.0  700.0
Cj  NaN       NaN  4.0  3.0  0.0  0.0  

In [16]:
#c
objective="max"
m=2#no of equations
n=4#no of variables
a=[[2,2,5,1,'<='],[3,3,4,0,"<="]]
b=[12,11]
Coeff=[3,3,2,1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   x3   x4   s1   s2   sol     ratio
1   0.0        s1  2.0  2.0  5.0  1.0  1.0  0.0  12.0  6.000000
2   0.0        s2  3.0  3.0  4.0  0.0  0.0  1.0  11.0  3.666667
Cj  NaN       NaN  3.0  3.0  2.0  1.0  0.0  0.0   0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0   0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s2
value of objective function is  11.0
simplex table:
    CBi basic_var   x1   x2        x3   x4   s1        s2        sol     ratio
1   0.0        s1  0.0  0.0  2.333333  1.0  1.0 -0.666667   4.666667  4.666667
2   3.0        x1  1.0  1.0  1.333333  0.0  0.0  0.333333   3.666667       inf
Cj  NaN       NaN  3.0  3.0  2.000000  1.0  0.0  0.000000   0.000000       NaN
Zj  NaN       NaN  3.0  3.0  4.000000  0.0  0.0  1.000000  11.000000       NaN
Lets proceed to next iteration
___________________________________________________

In [17]:
#d
objective="max"
m=3#no of equations
n=3#no of variables
a=[[2,3,0,'<='],[0,2,5,"<="],[3,2,4,"<="]]
b=[8,10,15]
Coeff=[3,5,4]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   x3   s1   s2   s3   sol     ratio
1   0.0        s1  2.0  3.0  0.0  1.0  0.0  0.0   8.0  2.666667
2   0.0        s2  0.0  2.0  5.0  0.0  1.0  0.0  10.0  5.000000
3   0.0        s3  3.0  2.0  4.0  0.0  0.0  1.0  15.0  7.500000
Cj  NaN       NaN  3.0  5.0  4.0  0.0  0.0  0.0   0.0       NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0   0.0       NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s1
value of objective function is  13.333333333333332
simplex table:
    CBi basic_var        x1   x2   x3        s1   s2   s3        sol     ratio
1   5.0        x2  0.666667  1.0  0.0  0.333333  0.0  0.0   2.666667       inf
2   0.0        s2 -1.333333  0.0  5.0 -0.666667  1.0  0.0   4.666667  0.933333
3   0.0        s3  1.666667  0.0  4.0 -0.666667  0.0  1.0   9.666667  2.416667
Cj  NaN       NaN  3.000000  5.0  4.0  0.000000  0.0  0.0   0.000000       NaN
Zj  

In [18]:
#d
objective="min"
m=3#no of equations
n=4#no of variables
a=[[2,2,5,1,'>='],[3,-3,4,0,">="],[4,-2,3,-1,">="]]
b=[43,11,25]
Coeff=[3,3,2,-1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   x3   x4   s1   s2   s3   sol  ratio
1   0.0        s1 -2.0 -2.0 -5.0 -1.0  1.0  0.0  0.0 -43.0   43.0
2   0.0        s2 -3.0  3.0 -4.0  0.0  0.0  1.0  0.0 -11.0   -inf
3   0.0        s3 -4.0  2.0 -3.0  1.0  0.0  0.0  1.0 -25.0  -25.0
Cj  NaN       NaN -3.0 -3.0 -2.0  1.0  0.0  0.0  0.0   0.0    NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x4
Leaving variable is  s1
value of objective function is  -43.0
simplex table:
    CBi basic_var   x1   x2   x3   x4   s1   s2   s3   sol  ratio
1   1.0        x4  2.0  2.0  5.0  1.0 -1.0 -0.0 -0.0  43.0  -43.0
2   0.0        s2 -3.0  3.0 -4.0  0.0  0.0  1.0  0.0 -11.0   -inf
3   0.0        s3 -6.0 -0.0 -8.0  0.0  1.0  0.0  1.0 -68.0  -68.0
Cj  NaN       NaN -3.0 -3.0 -2.0  1.0  0.0  0.0  0.0   0.0    NaN
Zj  NaN       NaN  2.0  2.0  5.0  1.0 -1.0  0.0  0.0  43.0    NaN
Lets

In [19]:
#e
objective="max"
m=3#no of equations
n=2#no of variables
a=[[2,3,'<='],[3,2,"<="],[1,1,">="]]
b=[30,24,3]
Coeff=[6,4]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   s1   s2   s3   sol  ratio
1   0.0        s1  2.0  3.0  1.0  0.0  0.0  30.0   15.0
2   0.0        s2  3.0  2.0  0.0  1.0  0.0  24.0    8.0
3   0.0        s3 -1.0 -1.0  0.0  0.0  1.0  -3.0    3.0
Cj  NaN       NaN  6.0  4.0  0.0  0.0  0.0   0.0    NaN
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0   0.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s3
value of objective function is  18.0
simplex table:
    CBi basic_var   x1   x2   s1   s2   s3   sol  ratio
1   0.0        s1  0.0  1.0  1.0  0.0  2.0  24.0   12.0
2   0.0        s2  0.0 -1.0  0.0  1.0  3.0  15.0    5.0
3   6.0        x1  1.0  1.0 -0.0 -0.0 -1.0   3.0   -3.0
Cj  NaN       NaN  6.0  4.0  0.0  0.0  0.0   0.0    NaN
Zj  NaN       NaN  6.0  6.0  0.0  0.0 -6.0  18.0    NaN
Lets proceed to next iteration
_______________________________________________________
Entering variable is  s3
Leaving varia